# Links

* Good Readings
    + Good reading about Standardization: https://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    + Standardization vs normalization: https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf
    + Scaling: https://scikit-learn.org/stable/modules/preprocessing.html
    + https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
    + How to evaluate the model: https://machinelearningmastery.com/evaluate-skill-deep-learning-models/
    + https://www.microsoft.com/en-us/research/blog/three-mysteries-in-deep-learning-ensemble-knowledge-distillation-and-self-distillation/
    + https://towardsdatascience.com/how-to-handle-large-datasets-in-python-with-pandas-and-dask-34f43a897d55
    + Batch size: https://machinelearningmastery.com/how-to-control-the-speed-and-stability-of-training-neural-networks-with-gradient-descent-batch-size/

# How to run: packages to install

    0- conda install nb_conda
    1- conda install matplotlib 
    2- conda install tensorflow
    3- conda install scikit-learn 
    4- conda install seaborn
        
    Run all initialization cells.
    After that select a dataset to load a learning model.
    Finally train and test it.

# Download and extract dataset

https://drive.google.com/file/d/1Jy3WVE0lNN08zU7xY0q-gN9DLnEPiBvD/view?usp=sharing

# Imports

In [ ]:
import json
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn
import sklearn
import sklearn.model_selection
import sklearn.preprocessing
import tensorflow as tf
import random
import pickle
from keras.callbacks import CSVLogger

# Hyperparameters

In [ ]:
# Neural networks are stochastic by design and that the source of randomness can be fixed to make results reproducible.
# Therefore, the most robust way to report results and compare models is to repeat your experiment many times (30+) and use summary statistics.
# Source: https://machinelearningmastery.com/reproducible-results-neural-networks-keras/
# Fix random seed.
#tf.random.set_seed(1234)
#np.random.seed(1234) # Scikit Learn does not have its own global random state but uses the numpy random state instead.

#CSV format
placementFeatures = ["#Cells", "#CellPins", "#Macros", "#MacroPins", "#PassingNets",
                     "TileArea", "CellDensity", "MacroDensity", "MacroPinDensity",
                     "Layer1BlkgDensity", "Layer2BlkgDensity", "Layer1PinDensity", "Layer2PinDensity"]

placementNeighborFeatures = ["NeighborTileArea", "NeighborCellArea", "NeighborL1PinArea", "NeighborL2PinArea",
                             "NeighborL1BlkArea", "NeighborL2BlkArea", "NeighborMacroArea",
                             "NeighborMacroPinArea", "#NeighborCells", "#NeighborCellPins", "#NeighborMacros",
                             "#NeighborMacroPins", "#NeighborPassingNets"]

GRFeatures = ["#VerticalOverflow", "#VerticalRemain", "#VerticalTracks",
              "#HorizontalOverflow", "#HorizontalRemain", "#HorizontalTracks"]

GRNeighborFeatures = ["#NeighborVerticalOverflow", "#NeighborVerticalRemain", "#NeighborVerticalTracks",
                      "#NeighborHorizontalOverflow", "#NeighborHorizontalRemain", "#NeighborHorizontalTracks"]

features = placementFeatures
features.extend(placementNeighborFeatures)
#features.extend(GRFeatures)
#features.extend(GRNeighborFeatures)

AllDRVTypes = ["AdjacentCutSpacing", "SameLayerCutSpacing", "EndOfLine", "FloatingPatch", "MinArea", "MinWidth",
  "NonSuficientMetalOverlap", "CutShort", "MetalShort", "OutOfDieShort", "CornerSpacing", "ParallelRunLength"]

SelectedDRVTypes = ["CutShort", "MetalShort"]

label_name = "HasDetailedRoutingViolation"

# Benchmarks
ispd18 = ["ispd18_test"+str(x) for x in range(1, 11)]
# ispd18.extend(["ispd18_test5_metal5", "ispd18_test8_metal5"])

ispd19 = ["ispd19_test"+str(x) for x in range(1, 11)]
# ispd19.extend(["ispd19_test7_metal5", "ispd19_test8_metal5", "ispd19_test9_metal5"])

circuits = ispd19
test_circuit = "ispd19_test10"
if test_circuit in circuits:
    circuits.remove(test_circuit)
circuits.remove("ispd19_test4")
circuits.remove("ispd19_test5")
circuits.remove("ispd19_test9")

#circuits = ["ispd19_test1"]
#test_circuit = "ispd19_test1"

# Paths
csv_path = "./data/FirstIterationGCellCoords/"
# csv_path = "./data/SecondIterationGCellCoords/"
# csv_path = "./data/FirstIterationFixedBin/"

# batch_size = 2048
# epochs = 300
# learning_rate = 0.0045 # decayed every two epochs using an exponential rate of 0.94;

batch_size = 32 # is important to ensure that each batch has a decent chance of containing a few positive samples
epochs = 20
learning_rate = 0.001 #Eh?Predictor=0.05, default=0.001
#beta = 0.001 #regularization
drop_out = 0.05 ##Eh?Predictor=0.05

METRICS = [tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'),
           tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc')]

matplotlib.rcParams['figure.figsize'] = (12, 10)
colors = matplotlib.pyplot.rcParams['axes.prop_cycle'].by_key()['color']

# Data preprocessing and graph functions

ATTENTION: If you want to deploy a model, it's critical that you preserve the preprocessing calculations.
The easiest way to implement them as layers, and attach them to your model before export.

In [ ]:
# The features will be rescaled so that they’ll have the properties of a standard normal distribution.
# mean (μ) = 0
# standard deviation (σ) = 1
def standardize(train_array, val_array, test_array=None):
    scaler = sklearn.preprocessing.StandardScaler()
    train_array = scaler.fit_transform(train_array)
    val_array = scaler.transform(val_array)
    if test_array is not None:
        test_array = scaler.transform(test_array)
        return train_array, val_array, test_array
    return train_array, val_array

# Claculate weight for classes
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
def calculate_class_weights(df, label_name):
    neg, pos = np.bincount(df[label_name])
    total = neg + pos
    print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0
    class_weight = {0: weight_for_0, 1: weight_for_1}
    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    return class_weight, neg, pos

# Some plot functions

In [ ]:
# plot the training loss and accuracy
def plot_train(history):   
    matplotlib.pyplot.style.use("ggplot")
    matplotlib.pyplot.figure()
    matplotlib.pyplot.plot(np.arange(0, epochs), history.history["loss"], label="train_loss")
    # matplotlib.pyplot.plot(np.arange(0, epochs), history.history["val_loss"], label="val_loss")
    # matplotlib.pyplot.plot(np.arange(0, epochs), history.history["accuracy"], label="train_acc")
    # matplotlib.pyplot.plot(np.arange(0, epochs), history.history["val_accuracy"], label="val_acc")
    matplotlib.pyplot.title("Training Loss")
    matplotlib.pyplot.xlabel("Epoch #")
    matplotlib.pyplot.ylabel("Loss/Accuracy")
    matplotlib.pyplot.legend(loc="lower left")
    matplotlib.pyplot.show()
    # matplotlib.pyplot.savefig("plot.png")

def plot_cm(labels, predictions, title=None, output_path=None, p=0.5):
    cm = sklearn.metrics.confusion_matrix(labels, predictions > p)
    matplotlib.pyplot.figure(figsize=(5,5))
    seaborn.heatmap(cm, annot=True, fmt="d")
    if title == None:
        matplotlib.pyplot.title('Confusion matrix')
    else:
        matplotlib.pyplot.title(title)
    matplotlib.pyplot.ylabel('Actual label')
    matplotlib.pyplot.xlabel('Predicted label')
    if output_path != None:
        matplotlib.pyplot.savefig(output_path)
    else:
        matplotlib.pyplot.show()
    
def calculate_metrics(model, results):
    m = {}
    for name, value in zip(model.metrics_names, results):
        m[name] = value
    if m['precision'] + m['recall'] != 0:
        f_score = (2 * m['precision'] * m['recall'])/(m['precision'] + m['recall'])
        m['F-score'] = f_score
    sqrt = math.sqrt((m['tp']+m['fp'])*(m['tp']+m['fn'])*(m['tn']+m['fp'])*(m['tn']+m['fn']))
    if sqrt != 0:
        mcc = (m['tp'] * m['tn'] - m['fp'] * m['fn'])/sqrt
        m['MCC'] = mcc
    return m

def print_metrics(metrics):
    print(int(metrics['tp']+metrics['fp']+metrics['tn']+metrics['fn']), int(metrics['fn']+metrics['tp']),
         int(metrics['fp']+metrics['tn']), int(metrics['tp']), int(metrics['tn']), int(metrics['fp']), int(metrics['fn']),
         round(metrics['precision'], 2), round(metrics['accuracy'], 2), round(metrics['F-score'], 2), round(metrics['MCC'], 2))
#     for x, y in metrics.items():
#         print(x,':', round(y, 2))
        

def print_train(history):
    print("loss:", history.history["loss"])
    print("accuracy:", history.history["accuracy"])
    print("tp:", history.history["tp"])
    print("fp:", history.history["fp"])
    print("tn:", history.history["tn"])
    print("fn:", history.history["fn"])
    print("precision:", history.history["precision"])
    print("recall", history.history["recall"])

# Load Training Data From ICCAD19

In [ ]:
test_df = pd.DataFrame()

dataframes = []
dataframes = [pd.read_csv(csv_path+circuit+".csv", dtype=np.float32) for circuit in circuits]
test_df = pd.read_csv(csv_path+test_circuit+".csv", dtype=np.float32)
    
#merge all DataFrames into a single one
df = pd.concat(dataframes, ignore_index=True)
#save some memory
dataframes.clear()

# Remove NodeIDs (debug info)
df = df.drop(columns=["NodeID"])
test_df = test_df.drop(columns=["NodeID"])

# Clear all DRV columns
df['HasDetailedRoutingViolation'] = False
test_df['HasDetailedRoutingViolation'] = False
# Filter for selected DRVs
for drv in SelectedDRVTypes:
    df['HasDetailedRoutingViolation'] = df['HasDetailedRoutingViolation'] | df[drv]
    test_df['HasDetailedRoutingViolation'] = test_df['HasDetailedRoutingViolation'] | test_df[drv]

df = df.drop(columns=AllDRVTypes)
test_df = test_df.drop(columns=AllDRVTypes)

# Remove GR info when not selected
if GRFeatures[0] not in features:
    df = df.drop(columns=GRFeatures)
    df = df.drop(columns=GRNeighborFeatures)
    test_df = test_df.drop(columns=GRFeatures)
    test_df = test_df.drop(columns=GRNeighborFeatures)

# Split 80/20 (train 80% test 20%)
train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)

# Build np arrays of labels and features.
train_labels = np.array(train_df.pop(label_name))
val_labels = np.array(val_df.pop(label_name))
test_labels = np.array(test_df.pop(label_name))
train_array = np.array(train_df)
val_array = np.array(val_df)
test_array = np.array(test_df)

# Scale
train_array, val_array, test_array = standardize(train_array, val_array, test_array)

# Claculate weight for classes
class_weight, neg, pos = calculate_class_weights(df, label_name)

# Load EhPredictor's dataset

In [ ]:
df = pd.read_csv("./data/ISPD14/EhPredictorISPD14.csv")

# drop l53 because is always zero
df.pop('l53')
df.pop('normal')

# Instead of having the number of shorts, use them as a boolean
df.loc[df['short'] > 0, 'short'] = 1

# Convert to log-space. l9 l43 l45 l52 l51
log_cols = ['l9', 'l43', 'l45', 'l52', 'l51']
eps=0.001 # 0 => 0.1¢
for col in log_cols:
    df[col] = np.log(df[col] + eps)

# CSV organization:
# des_perf_1_dataset=all_dataset[0:5476,:]
# des_perf_a_dataset=all_dataset[5476:16928,:]
# des_perf_b_dataset=all_dataset[16928:26928,:]
# fft_1_dataset=all_dataset[26928:28864,:]
# fft_2_dataset=all_dataset[28864:32113,:]
# fft_a_dataset=all_dataset[32113:38604,:]
# fft_b_dataset=all_dataset[38604:44375,:]
# matrix_mult_1_dataset=all_dataset[44375:52656,:]
# matrix_mult_a_dataset=all_dataset[52656:69168,:]
# matrix_mult_b_dataset=all_dataset[69168:90601,:]
# pci_bridge32_a_dataset=all_dataset[90601:94170,:]
# pci_bridge32_b_dataset=all_dataset[94170:103961,:]
# superblue11_a_dataset=all_dataset[103961:175113,:]
# superblue12_dataset=all_dataset[175113:241123,:]

# Test circuits: mgc fft_2
test_df = df.iloc[28864:32113]
df2 = df[0:28864]
df3 = df[32113:]
df = pd.concat([df2, df3])

# Use a utility from sklearn to split and shuffle our dataset.
train_df, val_df = sklearn.model_selection.train_test_split(df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('short'))
val_labels = np.array(val_df.pop('short'))
test_labels = np.array(test_df.pop('short'))

train_array = np.array(train_df)
val_array = np.array(val_df)
test_array = np.array(test_df)

# Scaling
train_array, val_array, test_array = standardize(train_array, val_array, test_array)

# Claculate weight for classes
class_weight, neg, pos = calculate_class_weights(df, 'short')

# DNN 50/50

In [ ]:
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = tf.keras.Sequential([tf.keras.layers.Dense(50, activation='relu'),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(50, activation='relu'),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

# Neural Network

In [ ]:
def make_model(metrics = METRICS, output_bias=None, lr=learning_rate):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    model = tf.keras.Sequential([tf.keras.layers.Dense(20,
                                                       activation='relu', # Relu throw away negative values
                                                       kernel_regularizer=tf.keras.regularizers.l2(beta)),
                                 tf.keras.layers.Dropout(drop_out),
                                 tf.keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

# Save/Load Model

In [ ]:
# initial_bias = np.log([pos/neg])
# model = make_model(output_bias = initial_bias)
# model.load_weights('path_to_file')

# model.save_weights('path_to_file')

# Train

In [ ]:
# Model weights are saved at the end of every epoch, if it's the best seen so far.
model_name = 'dnn50_50'
checkpoint_filepath = "./checkpoints/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath)

csv_logger = CSVLogger("model_history_log.csv", append=True)

dataset = tf.data.Dataset.from_tensor_slices((train_array, train_labels))
train_dataset = dataset.shuffle(len(train_array)).batch(batch_size)

initial_bias = np.log([pos/neg])

model = make_model(output_bias = initial_bias)
train_history = model.fit(train_dataset,
                          batch_size=batch_size,
                          validation_data=(val_array, val_labels),
                          class_weight=class_weight,
                          epochs=epochs,
                          callbacks=[checkpoint_callback, csv_logger])

with open('./checkpoints/history', 'wb') as file_pi:
    pickle.dump(train_history.history, file_pi)
model.save_weights('./checkpoints/'+model_name)

# Training performance

In [ ]:
plot_train(train_history)
print_train(train_history)

baseline_results = model.evaluate(train_array, train_labels, batch_size=batch_size, verbose=0)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)
train_predictions_baseline = model.predict(train_array, batch_size=batch_size)
plot_cm(train_labels, train_predictions_baseline, "Training Confusion Matrix")

# Test and check performance

In [ ]:
baseline_results = model.evaluate(test_array, test_labels, batch_size=batch_size, verbose=0)
metrics = calculate_metrics(model, baseline_results)
print_metrics(metrics)
test_predictions_baseline = model.predict(test_array, batch_size=batch_size)
plot_cm(test_labels, test_predictions_baseline, "Test Confusion Matrix")

# Save Predicted Node IDs to draw

In [ ]:
# Save all predited node ids
# This can be used to plot the violating bins inside C++ OpenROAD
violations = test_predictions_baseline > 0.5
violating_ids = []
i = 0
for x in zip (violations, test_df.iterrows()):
    if x[0]:
        violating_ids.append(int(x[1][0]))
with open(test_circuit+'violating_nodes.txt', 'w') as f:
    for item in violating_ids:
        f.write("%s\n" % item)